In [ ]:
import torch
import torch.nn as nn

### CLIP中文字数据的加载

In [1]:
# 传入参数
vocab_size : int
context_length : int
transformer_width : int
trasformer_heads : int
transformer_layer : int
vocab_size = vocab_size

# 初始化传入数据
context_length = context_length
vocab_size = vocab_size
token_embedding = nn.Embedding(vocab_size, transformer_width)
positional_embedding = nn.Parameter(torch.empty(context_length, transformer_width))

NameError: name 'vocab_size' is not defined

### dtype函数及具体的使用
dtype函数在这里会返回图像部分第一层卷积的weight权重数据
在这里dtype函数的用法更像固定调用，会固定调用出某层的数据类型

### type()函数用法
如果只有第一个参数则返回回想的类型， 三个参数会返回新的类型对象

In [ ]:
@property
def dtype(self):
    return self.visual.conv1.weight.dtype

### text处理中transformer的应用


In [ ]:
def build_attention_mask():
    # initialize square
    mask = torch.empty(context_length, context_length)
    mask.fill_(float("-inf"))
    mask.triu_(1) # zero out the lowe diagonal
    return mask

In [ ]:
class Transformer(nn.Module):
    def __init__(self, width:int,
                 layers:int,
                 heads:int,
                 attn_mask: torch.Tensor=build_attention_mask):
        super().__init__()
        self.width = width
        self.layers = layers
        self.resblocks = nn.Sequential(*[ResidualAttentionBlock(width, heads, attn_mask) for _ in range[layers]])

    def forward(self, x: torch.Tensor):
        return self.resblocks(x)

### LayerNorm层的应用

In [ ]:
class LayerNorm(nn.LayerNorm):
    def forward(self, x: torch.Tensor):
        orig_type = x.dtype
        ret = super().forward(x.type(torch.float32))
        return ret.type(orig_type)

### torch.arange 以及 torch.argmax
torch.argmax会输出矩阵最大值
torch.arange会输出给定步长的值
torch.arange(x.shape[0])会返回从[0,shape[0])的数组，默认步长为1

In [ ]:
transformer = Transformer(
    width=transformer_width,
    layers = transformer_layer,
    heads = trasformer_heads,
    attn_mask = None
)

ln_final = LayerNorm(transformer_width)
text_projection = nn.Parameter(torch.empty(context_length, transformer_width))

def encode_text(text):
    x = token_embedding(text).type(dtype) #[batch_size, n_ctx, d_model]
    x = x + positional_embedding.type(dtype)
    x = x.permute(1, 0, 2) # NLD -> LND
    x = transformer(x)
    x = x.permute(1, 0, 2) # LND -> NLD
    x = ln_final(x).type(dtype)

    # x.shape = [batch_size, n_ctx, transformer.width]
    # take features from the eot embedding (eot_token is the highest number in each sequence)
    x = x[torch.arange(x.shape[0]), text.argmax(dim=-1)] @ text_projection
    return x

### clip中图像数据的加载
clip以resnet和transformer分别作为主干网络，在这里我们使用resnet作为参考

In [ ]:
# 初始化
embed_dim: int
image_resolution: int
vision_layers: Union[Tuple[int, int, int, int], int]
vision_width: int
vision_patch_size: int

In [ ]:
class ModifiedResNet(nn.Module):
    def __init__(self, layers, output_dim, heads, input_resolution=224, width=64):
        super().__init__()
        self.output_dim = output_dim
        self.input_resolution = input_resolution

        self.conv1 = nn.Conv2d()

In [ ]:
vision_heads = vision_width * 32 // 64
visual = ModifiedResNet(
    layers = vision_layers,
    output_dim = embed_dim,
    heads = vision_heads,
    input_resolution = image_resolution,
    width = vision_width
)

def encode_imag(self, image):
    return self.visual(image.type(dtype))